In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_model_optimization as tfmot
model = tf.keras.models.load_model('DeepWeeds\my_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

data_root = 'DeepWeeds\\images\\images'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)
train = tf.concat([(x, y) for x, y in train_ds], axis=0)
images = tf.cast((train_ds)[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

converter.representative_dataset = representative_data_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# # converter.inference_input_type = tf.uint8  # or tf.uint8
# # converter.inference_output_type = tf.uint8  #
tflite_model = converter.convert()
open("weed_classifier_5.tflite", "wb").write(tflite_model)